

1. Model name: 
2. Domain: Multi-Criteria Recommender System (MCRS)

In [1]:
import random
import numpy as np
import pandas as pd
import torch
import time
from scipy.sparse import csr_matrix, save_npz, load_npz
from utils_torch import *
from scipy.linalg import expm
random.seed(2022)
np.random.seed(2022)

### 1. Load Dataset

In [3]:
import os
device = 'cuda:0'
#device = 'cpu'

current_directory = os.getcwd()
print("Current Directory:", current_directory)
# Configuration
dataset = "YM" # BA, YM, TA
path = f"{current_directory}/dataset/{dataset}"
cri_dict[dataset] =  5 

R_tr = []
R_ts = []

# Read csr matrix
for idx in range(cri_dict[dataset]):
    R_tr.append(csr2torch(load_npz(path + f'/{dataset}_tr_{idx}.npz')).to(device))
    R_ts.append(csr2torch(load_npz(path + f'/{dataset}_ts_{idx}.npz')).to(device))
    

n_users = R_tr[0].shape[0]
n_items = R_tr[0].shape[1]
n_cri = cri_dict[dataset]

print(f"number of users: {n_users}")
print(f"number of items: {n_items}")


ov_rating = torch.nonzero(R_tr[0]._values()).cpu().size(0) +torch.nonzero(R_ts[0]._values()).cpu().size(0)
edge = 0
for tr, ts in zip(R_tr, R_ts):    
    mc_ratings= torch.nonzero(tr._values()).cpu().size(0) +torch.nonzero(ts._values()).cpu().size(0)
    edge += mc_ratings
print(f"number of overall ratings: {ov_rating}")
print(f"number of MC ratings: {edge}")


Current Directory: /mnt/mlx-nfs/jdpark/cagf


/mnt/mlx-nfs/jdpark/cagf/utils_torch.py:99: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  indices = torch.LongTensor([coo_matrix.row, coo_matrix.col])


NameError: name 'cri_dict' is not defined

### 2. Run CA-GF

In [ ]:
start = time.time()
# Graph construction
MCEG = graph_construction(R_tr, cri_dict[dataset], device, version = 2)  # 0: overall, 1: MCEG
mceg_norm = normalize_sparse_adjacency_matrix(MCEG, 0.5)
MCEG = MCEG.to_dense()
P = mceg_norm.T @ mceg_norm

mode = 1
# Preference computation (optional)
if mode == True:
    X = R_tr[0].sum(axis=1).to_dense().view(-1, 1)
    for cri in range(1, n_cri):
        X = torch.concat((X, R_tr[cri].sum(axis=1).to_dense().view(-1, 1)), axis =1)
    K = normalize_sparse_adjacency_matrix_row(X, -1)
    C = normalize_sparse_adjacency_matrix_row(K.T @ K, -1)
    pref_mat  = K @ C

# Combinatorial Filtering  
ps = {0:1, 1:1.05, 2:1.8, 3:1} 
cri_mapping={0:2, 1:1, 2:1, 3:2, 4:0} 
predictions = []
for cri in range(cri_dict[dataset]):
    if mode == True:        
        predictions.append(torch.mul(pref_mat.T[cri].view(-1,1),inference_3(MCEG,P, ps, cri, n_users, cri_mapping[cri],device)))
    else:
        predictions.append(inference_3(MCEG,P, ps, cri, n_users, cri_mapping[cri],device))
prediction = sum(predictions)

print(f"Computing time:{time.time()-start}:.4f")


### 3. Evaluation

In [ ]:
results = prediction[:,:n_items]

results = results + (-99999)*MCEG.to_dense()[:n_users,:]
top_10_mat, rel10 = top_k(results, k = 10, device = device)
top_5_mat, rel5 = top_k(results, k = 5, device = device)
# Processing gt
if device == 'cpu':
    gt_mat = R_ts[0].to_dense()
else:
    gt_mat = R_ts[0].to_dense()
gt_mat[gt_mat<3] = 0
gt_mat[gt_mat>=3] = 1

_, i_rel10 =top_k(gt_mat, k = 10, device = device)
_, i_rel5 =top_k(gt_mat, k = 5, device = device)

# measure ranking-based evaluations
#print(f"Precision@5: {precision_k(top_5_mat, gt_mat, 5, device = device):.4f}, Precision@10: {precision_k(top_10_mat, gt_mat, 10, device = device):.4f} \n")
print(f"Recall@5: {recall_k(top_5_mat, gt_mat, 5, device = device):.4f}, Recall@10: {recall_k(top_10_mat, gt_mat, 10, device = device):.4f} \n")
print(f"NDCG@5: {ndcg_k(rel5, i_rel5, gt_mat, device = device):.4f} NDCG@10: {ndcg_k(rel10, i_rel10, gt_mat, device = device):.4f} \n")